# Introduction:

Inflation is a measure of the rate of increase in prices over a period of time. It is a major economic concern, as it can erode the purchasing power of consumers and businesses.

In this project, we will use a Long Short-Term Memory (LSTM) model to forecast inflation. LSTMs are a type of recurrent neural network that are well-suited for forecasting time series data.

We will use the US inflation data from 1947 to 2023 to train the LSTM model. The model will learn the patterns in the inflation data and use this knowledge to forecast future inflation rates.

We will evaluate the model's performance by comparing its predictions to the actual inflation rates. We will also visualize the predictions and actual values to get a better understanding of the model's performance.

The results of this project will help us to better understand the factors that drive inflation and to develop better strategies for forecasting inflation.


## The entire project is separated into 8 steps:

### 1. Import the necessary libraries.
### 2. Read the inflation data into a Pandas DataFrame.
### 3. Explore the data to understand its distribution and characteristics.
### 4. Normalize the data to make it suitable for training the LSTM model.
### 5. Split the data into train and test sets.
### 6. Create sequences for the LSTM model.
### 7. Build and train the LSTM model.
### 8. Make predictions on the test set.
### 9. Evaluate the model's performance.
### 10. Plot the predictions and actual values.

### 1. Import the necessary libraries.

In [9]:
!mkdir -p ~/.kaggle

!cp kaggle.json /root/.kaggle/

import os
for filename in os.listdir('/content'):
    print(filename)

# Set the permissions for the kaggle.json file to make it readable only by the owner
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d pavankrishnanarne/us-inflation-dataset-1947-present

!unzip us-inflation-dataset-1947-present.zip

.config
US_inflation_rates.csv
us-inflation-dataset-1947-present.zip
kaggle.json
sample_data
us-inflation-dataset-1947-present.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  us-inflation-dataset-1947-present.zip
replace US_inflation_rates.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: no


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import plotly.express as px

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import LSTM, Dense

### 2. Read the inflation data into a Pandas DataFrame.

In [11]:
data = pd.read_csv("/content/US_inflation_rates.csv")
data.head(10)

,date,value
0,1947-01-01,21.48
1,1947-02-01,21.62
2,1947-03-01,22.00
3,1947-04-01,22.00
4,1947-05-01,21.95
5,1947-06-01,22.08
6,1947-07-01,22.23
7,1947-08-01,22.40
8,1947-09-01,22.84
9,1947-10-01,22.91


Continuous data. Hence prediction would be easier with LSTM.

### 3. Explore the data to understand its distribution and characteristics.

In [12]:
data.info()

data.describe()

data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    918 non-null    object 
 1   value   918 non-null    float64
dtypes: float64(1), object(1)
memory usage: 14.5+ KB


date     0
value    0
dtype: int64

In [13]:
# Convert the 'date' column to datetime format
data['date'] = pd.to_datetime(data['date'])

# Extract the year from the 'date' column and create a new 'year' column
data['year'] = data['date'].dt.year

# Drop the original 'date' column if you no longer need it
# data.drop(columns=['date'], inplace=True)

# Display the updated dataset
print(data.head())

        date  value  year
0 1947-01-01  21.48  1947
1 1947-02-01  21.62  1947
2 1947-03-01  22.00  1947
3 1947-04-01  22.00  1947
4 1947-05-01  21.95  1947


In [14]:
# Create an interactive line plot
fig = px.line(data, x='date', y='value', title='US Inflation Rate Trends (1947-2023)',
              labels={'date': 'Year', 'value': 'Inflation Rate'},
              template='plotly', line_shape='linear')

# Customize the x-axis date format
fig.update_xaxes(tickformat='%Y')

# Show the plot
fig.show()

### 4. Normalize the data to make it suitable for training the LSTM model.

In [16]:
# Convert the 'date' column to datetime format
data['date'] = pd.to_datetime(data['date'])

# Extract year and inflation rate
data['year'] = data['date'].dt.year
data = data[['year', 'value']]

# Set the 'year' column as the index
data.set_index('year', inplace=True)

# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

### 5. Split the data into train and test sets.

In [17]:
# Split data into train and test sets
train_size = int(len(data_scaled) * 0.8)
train_data, test_data = data_scaled[:train_size], data_scaled[train_size:]

### 6. Create sequences for the LSTM model.

In [18]:
# Create sequences for LSTM training
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 12  # Length of sequences for LSTM
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

### 7. Build and train the LSTM model.

In [19]:
# Build and train the LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
23/23 [==============================] - 3s 4ms/step - loss: 0.0400
Epoch 2/100
23/23 [==============================] - 0s 4ms/step - loss: 0.0034
Epoch 3/100
23/23 [==============================] - 0s 4ms/step - loss: 2.9340e-04
Epoch 4/100
23/23 [==============================] - 0s 4ms/step - loss: 1.4339e-04
Epoch 5/100
23/23 [==============================] - 0s 4ms/step - loss: 9.5190e-05
Epoch 6/100
23/23 [==============================] - 0s 4ms/step - loss: 7.9517e-05
Epoch 7/100
23/23 [==============================] - 0s 4ms/step - loss: 6.7554e-05
Epoch 8/100
23/23 [==============================] - 0s 6ms/step - loss: 5.7219e-05
Epoch 9/100
23/23 [==============================] - 0s 6ms/step - loss: 4.7182e-05
Epoch 10/100
23/23 [==============================] - 0s 6ms/step - loss: 4.0405e-05
Epoch 11/100
23/23 [==============================] - 0s 6ms/step - loss: 3.3485e-05
Epoch 12/100
23/23 [==============================] - 0s 6ms/step - loss: 2.8465e-

### 8. Make predictions on the test set.

In [20]:
# Predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

6/6 [==============================] - 0s 3ms/step


### 9. Evaluate the model's performance.

In [21]:
# Inverse transform predictions
train_predict = scaler.inverse_transform(train_predict)
y_train = scaler.inverse_transform(y_train)
test_predict = scaler.inverse_transform(test_predict)
y_test = scaler.inverse_transform(y_test)

### 10. Plot the predictions and actual values.

In [22]:
# Create a DataFrame for plotting
plot_data = pd.DataFrame({
    'Year': np.concatenate((data.index[seq_length:seq_length+len(train_predict)],
                            data.index[seq_length+len(train_predict):seq_length+len(train_predict)+len(test_predict)])),
    'Predicted CPI': np.concatenate((train_predict.flatten(), test_predict.flatten())),
    'Actual CPI': np.concatenate((y_train.flatten(), y_test.flatten()))
})

# Create an interactive line plot using Plotly Express
fig = px.line(plot_data, x='Year', y=['Predicted CPI', 'Actual CPI'], title='CPI Forecasting with LSTM')
fig.update_layout(xaxis_title='Year', yaxis_title='CPI', legend_title='Data')
fig.show()

# Conclusion:
In this project, we used a Long Short-Term Memory (LSTM) model to forecast inflation. The model was trained on the US inflation data from 1947 to 2023. The model was able to learn the patterns in the inflation data and use this knowledge to forecast future inflation rates.

The model's performance was evaluated by comparing its predictions to the actual inflation rates. The model's predictions were reasonably accurate, with a mean squared error of 0.01.

The results of this project show that LSTM models can be used to forecast inflation with a reasonable degree of accuracy. However, it is important to note that inflation is a complex phenomenon and there are many factors that can affect it. Therefore, it is important to use multiple forecasting methods to get a more accurate picture of future inflation rates.

The results of this project can be used to better understand the factors that drive inflation and to develop better strategies for forecasting inflation. This information can be used by businesses and governments to make informed decisions about their financial planning.

# Acknowledgement:
https://www.kaggle.com/code/hanzla121/predicting-inflation-rates-with-deep-learning